In [2]:
import datetime as dt
import mysql.connector as connector
connection = connector.connect(user="root",password="")
cursor = connection.cursor()
cursor.execute("USE little_lemon")

In [3]:
def action(sql):
    cursor.execute(sql);
    col = cursor.column_names
    results = cursor.fetchall()
    print(col)
    for result in results:
        print(result)

In [4]:
action("""SELECT * FROM bookings""")

('BookingID', 'TableNo', 'GuestFirstName', 'GuestLastName', 'BookingSlot', 'EmployeeID')
(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1)
(2, 11, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 6)
(3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3)
(4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4)
(5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2)
(6, 10, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5)


In [5]:
action("SELECT * FROM Orders")

('OrderID', 'TableNo', 'MenuID', 'BookingID', 'BillAmount', 'Quantity')
(1, 12, 1, 1, 86, 2)
(2, 19, 2, 2, 37, 1)
(3, 15, 2, 3, 37, 1)
(4, 5, 3, 4, 40, 1)
(5, 8, 1, 5, 43, 1)


In [8]:
query1 = """SELECT B.BookingID,CONCAT(B.GuestFirstName," ",B.GuestLastName),O.BillAmount FROM
            bookings B INNER JOIN Orders O
            ON B.BookingID = O.BookingID"""
print(query1)

SELECT B.BookingID,CONCAT(B.GuestFirstName," ",B.GuestLastName),O.BillAmount FROM
            bookings B INNER JOIN Orders O
            ON B.BookingID = O.BookingID


In [9]:
action(query1)

('BookingID', 'CONCAT(B.GuestFirstName," ",B.GuestLastName)', 'BillAmount')
(1, 'Anna Iversen', 86)
(2, 'Joakim Iversen', 37)
(3, 'Vanessa McCarthy', 37)
(4, 'Marcos Romero', 40)
(5, 'Hiroki Yamane', 43)


In [13]:
ex_stored_procedure = """
CREATE PROCEDURE GETBillBYBookingID(IN BID INT,OUT BAMOUNT INT)
BEGIN
    SELECT O.BillAmount INTO BAMOUNT
    FROM Orders O
    INNER JOIN bookings B
    ON B.BookingID = O.BookingID
    WHERE O.BookingID = BID;
END
"""
print(ex_stored_procedure)


CREATE PROCEDURE GETBillBYBookingID(IN BID INT,OUT BAMOUNT INT)
BEGIN
    SELECT O.BillAmount INTO BAMOUNT
    FROM Orders O
    INNER JOIN bookings B
    ON B.BookingID = O.BookingID
    WHERE O.BookingID = BID;
END



In [14]:
cursor.execute(ex_stored_procedure)

In [26]:
BID = 1
cursor.execute("CALL GETBillBYBookingID({},@BAMOUNT)".format(BID))

In [27]:
cursor.execute("SELECT @BAMOUNT")

In [28]:
data = cursor.fetchall()

In [29]:
data

[(86,)]

In [32]:
query2 = """
CREATE PROCEDURE CUPONCUSTOMER()
BEGIN
SELECT B.BookingID AS BookingID,CONCAT(B.GuestFirstName," ",B.GuestLastName)
AS FullName,O.BillAmount FROM bookings B
INNER JOIN orders O
ON B.BookingID = O.BookingID
WHERE O.BillAmount >= 50
ORDER BY BillAmount DESC;
END
"""
print(query2)


CREATE PROCEDURE CUPONCUSTOMER()
BEGIN
SELECT B.BookingID AS BookingID,CONCAT(B.GuestFirstName," ",B.GuestLastName)
AS FullName,O.BillAmount FROM bookings B
INNER JOIN orders O
ON B.BookingID = O.BookingID
WHERE O.BillAmount >= 50
ORDER BY BillAmount DESC;
END



In [33]:
cursor.execute(query2)

In [34]:
cursor.callproc('CUPONCUSTOMER')

()

In [35]:
cursor

In [37]:
cursor.stored_results()

<generator object CMySQLCursor.stored_results at 0x000001B7381FB370>

In [38]:
## it is a generator object
## we call next to get the result
results = next(cursor.stored_results())

In [39]:
results

In [40]:
data = results.fetchall()

In [41]:
for item in data:
    print(item)

(1, 'Anna Iversen', 86)


In [42]:
#or
cursor.execute("CALL CUPONCUSTOMER")

In [43]:
data = cursor.fetchall()

In [45]:
for item in data:
    print(item)

(1, 'Anna Iversen', 86)


# TASK 1
Little Lemon is running a marketing campaign this month. They need to issue a discount coupon to the top spender on daily bases. 

Create a stored procedure TopSpender that can retrieve the following:

the booking ID, 

guest’s full name, 

and the bill amount of the top spender at closing. 

Call the procedure and print the results.